In [1]:
# import libraries
import re
import os
import numpy as np
import pandas as pd
import requests
import json 
from pandas.io.json import json_normalize
from requests.auth import HTTPBasicAuth
from getpass import getpass
import this

os.getcwd()
os.listdir()

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


['multiTimeline.csv',
 'tweetscount.ipynb',
 'virus.ipynb',
 'multiTimeline2.csv',
 'README.md',
 'virusds.csv',
 'geoMap.csv',
 'google_trend_country.ipynb',
 '.ipynb_checkpoints',
 'virus-Copy1.ipynb']

In [2]:
# to get pandas to show 999 rows of the dataframes
#pd.options.display.max_rows = 999

In [3]:
# loading ds into dataframe
virus = pd.read_csv("virusds.csv")

In [4]:
# documentation 
"""
Sno —> Serial number —> int
Date —> Date and time of observation —> date
Province/State —> Province or State of observation —> str
Country —> Country of observation —> str
Last Update —> Last update date of the row in UTC -> date
Confirmed —> Number of confirmed cases —> decimal
Deaths —> Number of deaths —> decimal
Recovered —> Number of recovered cases —> decimal
"""
virus.head(10)

,Sno,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020 12:00:00,Anhui,China,2020-01-22 12:00:00,1.0,0.0,0.0
1,2,01/22/2020 12:00:00,Beijing,China,2020-01-22 12:00:00,14.0,0.0,0.0
2,3,01/22/2020 12:00:00,Chongqing,China,2020-01-22 12:00:00,6.0,0.0,0.0
3,4,01/22/2020 12:00:00,Fujian,China,2020-01-22 12:00:00,1.0,0.0,0.0
4,5,01/22/2020 12:00:00,Gansu,China,2020-01-22 12:00:00,0.0,0.0,0.0
5,6,01/22/2020 12:00:00,Guangdong,China,2020-01-22 12:00:00,26.0,0.0,0.0
6,7,01/22/2020 12:00:00,Guangxi,China,2020-01-22 12:00:00,2.0,0.0,0.0
7,8,01/22/2020 12:00:00,Guizhou,China,2020-01-22 12:00:00,1.0,0.0,0.0
8,9,01/22/2020 12:00:00,Hainan,China,2020-01-22 12:00:00,4.0,0.0,0.0
9,10,01/22/2020 12:00:00,Hebei,China,2020-01-22 12:00:00,1.0,0.0,0.0


In [5]:
virus.shape

(700, 8)

In [6]:
virus.describe()

,Sno,Confirmed,Deaths,Recovered
count,700.00000,700.000000,700.000000,700.000000
mean,350.50000,141.224286,3.077143,3.407143
std,202.21688,859.834237,27.759694,22.586377
min,1.00000,0.000000,0.000000,0.000000
25%,175.75000,2.000000,0.000000,0.000000
50%,350.50000,8.000000,0.000000,0.000000
75%,525.25000,55.250000,0.000000,1.000000
max,700.00000,13522.000000,414.000000,396.000000


In [7]:
# checking how prevalent missing values are in the data
# defining a function to check null values and null values %:
def null_cols(ds):
    """
    Checks whether the value in each field is missing (null) and return either 
    True or False for each field, totaling up the number of True values by column.

    Then does the same, but returns the value as a percentage. Useful to decide where 
    to drop.
    """
    
    return ds.isnull().sum(), ds.isna().mean().round(4) * 100

In [8]:
nulls_abs, nulls_rel = null_cols(virus)

In [9]:
# 164 in Provice/State
nulls_abs

Sno                 0
Date                0
Province/State    164
Country             0
Last Update         0
Confirmed           0
Deaths              0
Recovered           0
dtype: int64

In [10]:
# Province/State 23,43% null valls
nulls_rel

Sno                0.00
Date               0.00
Province/State    23.43
Country            0.00
Last Update        0.00
Confirmed          0.00
Deaths             0.00
Recovered          0.00
dtype: float64

In [11]:
# Checking Unique entries per colum:
[col + ": " + str(len(virus[col].unique())) + " uniques" for col in virus]

['Sno: 700 uniques',
 'Date: 13 uniques',
 'Province/State: 57 uniques',
 'Country: 31 uniques',
 'Last Update: 92 uniques',
 'Confirmed: 183 uniques',
 'Deaths: 15 uniques',
 'Recovered: 36 uniques']

In [12]:
# Checking Unique "Countries"
virus["Country"].unique()

array(['China', 'US', 'Japan', 'Thailand', 'South Korea',
       'Mainland China', 'Hong Kong', 'Macau', 'Taiwan', 'Singapore',
       'Philippines', 'Malaysia', 'Vietnam', 'Australia', 'Mexico',
       'Brazil', 'France', 'Nepal', 'Canada', 'Cambodia', 'Sri Lanka',
       'Ivory Coast', 'Germany', 'Finland', 'United Arab Emirates',
       'India', 'Italy', 'Sweden', 'Russia', 'Spain', 'UK'], dtype=object)

In [13]:
# duplicating the data
c_virus = virus.copy()

# correcting China entries
c_virus["Country"] = c_virus["Country"].str.replace('Mainland China', 'China')
c_virus["Country"] = c_virus["Country"].str.replace('Hong Kong', 'China')
#c_virus["Country"] = c_virus["Country"].str.replace('Taiwan', 'China') // We decided to 
# keep Taiwan as a Country
c_virus["Country"] = c_virus["Country"].str.replace(' China', 'China')
c_virus["Country"] = c_virus["Country"].str.replace('Macau', 'China')
c_virus["Country"].unique()

array(['China', 'US', 'Japan', 'Thailand', 'South Korea', 'Taiwan',
       'Singapore', 'Philippines', 'Malaysia', 'Vietnam', 'Australia',
       'Mexico', 'Brazil', 'France', 'Nepal', 'Canada', 'Cambodia',
       'Sri Lanka', 'Ivory Coast', 'Germany', 'Finland',
       'United Arab Emirates', 'India', 'Italy', 'Sweden', 'Russia',
       'Spain', 'UK'], dtype=object)

In [14]:
# Correcting 'Taiwan as a country' (1 entry)
c_virus[(c_virus["Province/State"] == "Taiwan") & (c_virus["Country"] != "Taiwan")]
c_virus.loc[(c_virus["Province/State"] == "Taiwan") & (c_virus["Country"] != "Taiwan"), "Country"] = "Taiwan"

In [15]:
c_virus[(c_virus["Province/State"] == "Taiwan") & (c_virus["Country"] != "Taiwan")]

,Sno,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered


In [16]:
# checking the unique Last Updates
c_virus["Last Update"].unique()

array(['2020-01-22 12:00:00', '2020-01-23 12:00:00',
       '2020-01-24 12:00:00', '2020-01-24 16:00:00',
       '2020-01-25 22:00:00', '2020-01-25 12:00:00',
       '2020-01-26 23:00:00', '2020-01-27 20:30:00',
       '2020-01-28 23:00:00', '2020-01-28 18:00:00',
       '2020-01-29 21:00:00', '2020-01-30 21:30:00',
       '2020-01-31 19:00:00', '2020-01-02 23:33:00',
       '2020-02-02 02:13:00', '2020-02-02 01:23:00',
       '2020-02-02 00:53:00', '2020-02-02 01:53:00',
       '2020-02-02 01:33:00', '2020-01-02 23:43:00',
       '2020-02-02 01:13:00', '2020-02-02 01:03:00',
       '2020-02-02 03:05:00', '2020-02-02 00:23:00',
       '2020-02-02 03:43:00', '2020-02-02 03:23:00',
       '2020-02-02 02:53:00', '2020-01-02 15:53:00',
       '2020-01-02 15:23:00', '2020-02-02 02:03:00',
       '2020-01-02 15:43:00', '2020-02-02 01:43:00',
       '2020-02-02 02:33:00', '2020-02-02 00:43:00',
       '2020-01-02 18:53:00', '2020-01-31 10:37:00',
       '2020-02-02 02:23:00', '2020-01-31 15:2

In [17]:
# Removing Time (hh:mm:ss) from the 'Last Update' col
def time_eater(entity):
    """
    Checks if an entity has a time after a date, and removes the time.
    """
    return re.sub("\s+\d{1,2}\:\d{2}:\d{2}", "", entity)

In [18]:
# applies the time_eater function to the 'Last Update' col to clean the time
c_virus["Last Update"] = c_virus["Last Update"].apply(time_eater)


In [19]:
c_virus["Last Update"].unique()

array(['2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25',
       '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29',
       '2020-01-30', '2020-01-31', '2020-01-02', '2020-02-02',
       '2020-03-02', '2020-04-02'], dtype=object)

In [20]:
# Clean 'Date' as done for 'Last Update'
time_eater(c_virus["Date"][0])

c_virus["Date"] = c_virus["Date"].apply(time_eater)

In [25]:
# Need to change the format of 'Date' so that it has 'Last Update' (YYYY-MM-DD)
c_virus["Date"] = pd.to_datetime(c_virus["Date"])

# Setting 'Last Update' in Timestamp format as well
c_virus["Last Update"] = pd.to_datetime(c_virus["Last Update"])

In [26]:
# After analysis, we reached the decision of dropping the 'Last Update', since the 'Dates'
# correspond to real world dates.
c_virus.drop("Last Update", axis = 1, inplace = True)

In [44]:
# Now to drop the rows from 03/02/2020 onwards
drop_late_dates = c_virus[c_virus["Date"] > "02/02/2020"]
c_virus.drop(drop_late_dates.index, axis = 0, inplace = True)

In [45]:
c_virus

,Sno,Date,Province/State,Country,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,China,1.0,0.0,0.0
1,2,01/22/2020,Beijing,China,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,China,6.0,0.0,0.0
3,4,01/22/2020,Fujian,China,1.0,0.0,0.0
4,5,01/22/2020,Gansu,China,0.0,0.0,0.0
...,...,...,...,...,...,...,...
626,627,02/02/2020,"Boston, MA",US,1.0,0.0,0.0
627,628,02/02/2020,"Los Angeles, CA",US,1.0,0.0,0.0
628,629,02/02/2020,"Orange, CA",US,1.0,0.0,0.0
629,630,02/02/2020,"Seattle, WA",US,1.0,0.0,0.0


# IGNORE

In [21]:
"""# Check the 'Last Update' against 'Date' —> 179 cases
c_virus[(c_virus["Date"] != c_virus["Last Update"])]

# Check entities where 'Last Update' comes AFTER the 'Date' -> 81 cases
c_virus[(c_virus["Date"] < c_virus["Last Update"])]

# Check entities where 'Last Update' come BEFORE the 'Date' -> 98 cases 
c_virus[(c_virus["Date"] > c_virus["Last Update"])]"""

,Sno,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
497,498,2020-02-01,Hubei,China,2020-01-02,9074.0,294.0,215.0
504,505,2020-02-01,Chongqing,China,2020-01-02,262.0,1.0,3.0
515,516,2020-02-01,Yunnan,China,2020-01-02,93.0,0.0,2.0
516,517,2020-02-01,Liaoning,China,2020-01-02,64.0,0.0,1.0
518,519,2020-02-01,Shanxi,China,2020-01-02,56.0,0.0,1.0
...,...,...,...,...,...,...,...,...
695,696,2020-02-03,"Boston, MA",US,2020-01-02,1.0,0.0,0.0
696,697,2020-02-03,"Los Angeles, CA",US,2020-01-02,1.0,0.0,0.0
697,698,2020-02-03,"Orange, CA",US,2020-01-02,1.0,0.0,0.0
698,699,2020-02-03,"Seattle, WA",US,2020-01-02,1.0,0.0,0.0


In [23]:
"""# The way I'm thinking, when 'Last Update' comes AFTER the 'Date' it's because new 
# information appeared/was corrected about the 'Date' it's refering to, while when it comes
# BEFORE it's because the month and the day are reversed; I think they are all for the 01-02
# and they should be for the 02-01 in China and US?

# 38
len(c_virus[(c_virus["Date"] > c_virus["Last Update"]) & (c_virus["Last Update"] != "2020-01-02")])

# 60
c_virus[(c_virus["Date"] > c_virus["Last Update"]) & (c_virus["Last Update"] == "2020-01-02")]"""

,Sno,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
497,498,2020-02-01,Hubei,China,2020-01-02,9074.0,294.0,215.0
504,505,2020-02-01,Chongqing,China,2020-01-02,262.0,1.0,3.0
515,516,2020-02-01,Yunnan,China,2020-01-02,93.0,0.0,2.0
516,517,2020-02-01,Liaoning,China,2020-01-02,64.0,0.0,1.0
518,519,2020-02-01,Shanxi,China,2020-01-02,56.0,0.0,1.0
520,521,2020-02-01,Gansu,China,2020-01-02,40.0,0.0,0.0
526,527,2020-02-01,NaN,Japan,2020-01-02,20.0,0.0,1.0
532,533,2020-02-01,Qinghai,China,2020-01-02,9.0,0.0,0.0
533,534,2020-02-01,NaN,Germany,2020-01-02,8.0,0.0,0.0
536,537,2020-02-01,NaN,France,2020-01-02,6.0,0.0,0.0


pandas._libs.tslibs.timestamps.Timestamp

In [ ]:
""""# TODO: Correct 'Last Update' month & or day
# TODO: Drop rows where 'Last Update' is after 02/02/202

#c_virus['Diff'] = (c_virus['Confirmed'] - c_virus['Confirmed'].shift(-1)) / c_virus['Confirmed']
# (row.iloc[0]-row.iloc[1])/row.iloc[0]*100
# c_virus["Percentage Change"] = c_virus["Confirmed"].pct_change()
c_virus""""

In [ ]:
"""multi_group = c_virus.groupby(["Date","Country"])[["Confirmed","Deaths"]].max()#.reset_index()"""

"""
multi_group2=[i.split(" ")[0] for i in virus["Date"]]
multi_group_series = pd.Series( (v for v in multi_group2) )
multi_group["Date"]=multi_group_series
multi_group.head(50)

"""

#new_row_df=pd.DataFrame([["01/22-2020","Worldwide"]])
#virusconcat = pd.concat([multi_group.loc[0:38],new_row_df])
#virusconcat

"""
multi_group["Date"] = pd.to_datetime(multi_group["Date"])
multi_group
#multi_group.set_index(pd.unique(pd.Series(multi_group["Date"])))
multi_group

multi_group["Date"].unique()
"""
"""multi_group"""

In [ ]:
"""# Adding a condition that will filter the data and show us only columns where the number 
# of null values were greater than zero for each dataset:

nulls_abs[nulls_abs > 0]"""

In [ ]:
""""
DON'T RUN THIS CELL YET!!!
"""

"""
# Judgement call: droping information that we don't think it's going to be very useful 
# to our analysis (removing those columns from your datasets) with the drop method.
# We will add these column names to a list, and then we will pass those columns to the 
# drop method and indicate that we want columns (not rows) dropped by setting the axis 
# parameter to 1.

# defining a function to create a list:
def drop_cols(bad_cols):
    """#Adding col names to a list to be droped; in this case as long as the col has a 
    #single null value in it, since, in this case, if it has one, their all null."""
    #return list(bad_cols[bad_cols > 0].index)

# Droping the columns with null values is (always?) a judgment call. 
# In a discussion on analyticsvidhya.com 
# (what-should-be-the-allowed-percentage-of-missing-values/2456), the sugestion is to drop
# after the 30% ceiling has been reached. Still, it will depend.

# For starters I'll check what entities have the missing values and what don't;
#missing = merged.loc[(merged["ViewCount"].notna() != True)]
"""

In [ ]:
test_ds = virus.copy()

In [ ]:
test_ds.drop("Date", axis = 1, inplace = True)

In [ ]:
test_ds.drop_duplicates()

In [ ]:
test_ds["Last Update"].unique()